# MongoDB basic test functions
Tous les fonctions de tests sont écrits dans le fichier `../TimeSeriesTools/mongodb_test.py`. 
<br>Pour exécuter la fonction à tester, il faut utiliser les lignes commandes dans un terminal ou exécuter les cellules avec les paramètres d'entrée dans le fichier `../MongoDBNotebook/mongoDB_inference_test.ipynb` avec jupyter-notebook. 

#### Liste des paramètres d'entrée : 
 - `--query`: le string de la requête
   - valeur par défaut : '{"timestamp":{"$regex":"10:"}}'
 - `--value`: la requête des modifications des données sélectionnées
    - valeur par défaut : '{"$set":{"timestamp":"00/00/0000 00:00"}}
 - `--topic`: indiquer le nom du topic désiré
    - valeur par défaut : "eolienne_DT"
 - `--collection`: indiquer le nom de la collection désirée
    - valeur par défaut : "eolienne_DT"
 - `--tracer`: indiquer le nom du tracer à initialiser
    - valeur par défaut : "mongodb_test_eolienne_1_jour"
 - `--domain`: indiquer l'address du domaine pour connecter à la base de mongodb
    - valeur par défaut : "localhost"
 - `--port`: indiquer la porte pour connecter à la base de mongodb
    - valeur par défaut : "27017"
 - `--function`: indiquer le nom de la fonction à tester, il n'a pas de valeur par défaut donc cet paramètre est **obligatoire**. 

**Remarque** : 
- Sauf que le numéro de la porte est int, toutes les types des paramètres d'entrée sont string.
- Les paramètres suivantes sont optionnels pour toutes les functions : 
    - <--tracer>
    - <--collection>
    - <--domain>
    - <--port>

Dans le script python il y a 6 fonctions de test à choisir :
- `test_insert_bulk` : Mesurer le temps pour insérer toutes les données collectées dans la base de donnée à la fois.
  - paramètres optionnels: <--topic>  

- `test_insert_one` : Mesurer le temps pour insérer toutes les données collectées dans la base de donnée ligne par ligne.
  - paramètres optionnels: <--topic>
 
- `find_some_data` : Mesurer le temps pour trouver les données correspondantes à la requête dans la base de donnée.
  - paramètres optionnels: <--query>
 
- `find_all_data` : Mesurer le temps pour lire toutes les données dans la base de donnée.

- `update_some_data` : Mesurer le temps pour modifier les données correspondantes à la requête dans la base de donnée.
  - paramètres optionnels: <--query>,<--value>

- `update_all_data`: Mesurer le temps pour modifier toutes les données dans la base de donnée.
  - paramètres optionnels: <--query>,<--value>
  

### Dependence : service kafka, kafka zookeeper, service mongoDB, jaeger-all-in-one, utils.py, mongodb_test.py

Après l'exécution du script, un tracer qui contient des sous-span (chaque span précise une étape d'opération) est signalé. Les informations et les barres intéractives sont ensuite disponibles sur wen UI de jeager (http://localhost:16686 par défaut).

## Insertion test :
Avant les tests d'insertion, il est obligatoire d'avoire les données dans un topic parce que les codes utilisent un kafka consumer comme la ressource de données. Si les données sont pas encore été envoyées, utiliser le script ../MongoDBNotebook/producer_jour_1_DT.ipynb pour envoyer les données d'éolienne (utiliser les fichiers corrigés comme ressource) pour tester.

### Insert data from a topic line by line. 
Cette fonction contient deux sous-span : l'un pour la collection des données venant d'un topic de kafka, et l'autre pour l'insertion des données collectées ligne par ligne.<br>Dans la prémière étape, le script lit les données avec un kafka consumer, restructure au format de dictionnaire et enregistre les dictionnaires dans une liste. Une ligne de donnée incorrecte est insérée dans la liste pour vérifier la validation de données.<br>Dans la deuxième étape la liste de données est insérée dans la base de mongodb ligne par ligne en utilisant une boucle pour tester la délai d'insertion. 

#### Code exemple : 

In [ ]:
%run ../TimeSeriesTools/mongodb_test.py --function test_insert_one --topic "eolienne_DT" --collection "eolienne_DT" --domain "localhost" --port 27017 --tracer "test_insert_lines"

### Insert data from a topic in bulk. 
Cette fonction contient deux sous-span : l'un pour la collection des données comme ci-dessus et l'autre pour l'insertion des données collectées à la fois.<br>Dans la deuxième étape la liste de données est insérée dans la base de mongodb à la fois en utilisant tester la délai d'insertion. 

In [ ]:
%run ../TimeSeriesTools/mongodb_test.py --function test_insert_bulk

## Read test : 
### Read all data from a mongodb collection.

Cette fonction contient un seul span pour la lecture de toutes les données dans une collection. Après l'exécution du code, le nombre total des données retrouvées est retourné et écrit dans le log de span.

Code exemple :

In [ ]:
%run ../TimeSeriesTools/mongodb_test.py --function find_all_data --port 27017 --collection "eolienne_DT"

### Select some data from a mongodb collection by a query. 

Cette fonction contient un seul span pour la lecture des données désirées dans une collectionen utilisant un filtrage (un requête). Après l'exécution du code, le nombre total des données retrouvées est retourné et écrit dans le log de span.

In [ ]:
%run ../TimeSeriesTools/mongodb_test.py --function find_some_data --query '{"Heure":{"$regex":"11:"}}' --collection "eolienne_DT"

## Update test :
### Update all documents in a mongodb collection.

Cette fonction contient un seul span pour la modification pour toutes les données retrouvées dans une collectionen. Après l'exécution du code, le nombre total des données modifiées est retourné et écrit dans le log de span.<br> 
Utiliser le paramètre --value pour préciser le requête afin de changer les valeurs des champs désirés.

In [ ]:
%run ../TimeSeriesTools/mongodb_test.py --function update_all_data --value '{"$set":{"C2H6":"0"}}'

### Update selected documents in a mongodb collection. 

Cette fonction contient un seul span pour la modification des données retrouvées dans une collectionen utilisant un filtrage (un requête). Après l'exécution du code, le nombre total des données modifiées est retourné et écrit dans le log de span.<br>
Utiliser le paramètre --query pour sélectionner les données à modifier et le paramètre --value pour préciser le requête afin de changer les valeurs des champs désirés.

In [ ]:
%run ../TimeSeriesTools/mongodb_test.py --function update_some_data --query '{"Heure":{"$regex":"02/10"}}' --value '{"$set":{"C2H6":"1"}}'